In [69]:
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
import numpy as np
import logging
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
import matplotlib.dates as mdates
import os

In [70]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set the base path to the desired directory on Google Drive
base_path = '/content/drive/MyDrive/Study_1_Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
def read_csv(file_path):
    data = pd.read_csv(file_path)
    return data

In [72]:
def process_data(data, columns_to_remove):
    processed_data = data.drop(columns=columns_to_remove).values
    return processed_data

In [73]:

def construct_3d_array(base_dir, participants, simulations, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye):
    """
    Construct 3D array from CSV files.
    """
    num_rows = 180  # Define number of rows to keep (last 180 rows)
    arrays_3d = []

    for participant in participants:
        participant_id = f"{int(participant):02d}"  # Format participant number to two digits

        valid_simulations = []

        for simulation in simulations:
            hr_file_path = os.path.join(base_dir, participant_id, simulation, f'HR{simulation.capitalize()}.csv')
            gsr_file_path = os.path.join(base_dir, participant_id, simulation, f'EDA{simulation.capitalize()}_downsampled.csv')
            head_file_path = os.path.join(base_dir, participant_id, simulation, 'head_tracking_downsampled.csv')
            eye_file_path = os.path.join(base_dir, participant_id, simulation, 'eye_tracking_downsampled.csv')

            # Check if all files exist
            if all(os.path.exists(file) for file in [hr_file_path, gsr_file_path, head_file_path, eye_file_path]):
                valid_simulations.append(simulation)

        num_valid_simulations = len(valid_simulations)
        if num_valid_simulations == 0:
            continue  # Skip this participant if no valid simulations are found

        array_3d = np.zeros((num_valid_simulations, num_rows, 47)) # hr=1, gsr=1, head=15-3, eye=41-8 total columns after removing columns= 48

        for s_idx, simulation in enumerate(valid_simulations):
            # Process hr data
            hr_file_path = os.path.join(base_dir, participant_id, simulation, f'HR{simulation.capitalize()}.csv')
            hr_data = read_csv(hr_file_path)
            processed_hr_data = process_data(hr_data, columns_to_remove_hr)
            processed_hr_data = processed_hr_data[-num_rows:]  # Keep only the last 180 rows

            # Process gsr data
            gsr_file_path = os.path.join(base_dir, participant_id, simulation, f'EDA{simulation.capitalize()}_downsampled.csv')
            gsr_data = read_csv(gsr_file_path)
            processed_gsr_data = process_data(gsr_data, columns_to_remove_gsr)
            processed_gsr_data = processed_gsr_data[-num_rows:]  # Keep only the last 180 rows

            # Process head data
            head_file_path = os.path.join(base_dir, participant_id, simulation, 'head_tracking_downsampled.csv')
            head_data = read_csv(head_file_path)
            processed_head_data = process_data(head_data, columns_to_remove_head)
            processed_head_data = processed_head_data[-num_rows:]  # Keep only the last 180 rows

            # Process eye data
            eye_file_path = os.path.join(base_dir, participant_id, simulation, 'eye_tracking_downsampled.csv')
            eye_data = read_csv(eye_file_path)
            processed_eye_data = process_data(eye_data, columns_to_remove_eye)
            processed_eye_data = processed_eye_data[-num_rows:]  # Keep only the last 180 rows

            # Combine processed data
            combined_data = np.concatenate((processed_hr_data, processed_gsr_data, processed_head_data, processed_eye_data), axis=1)



            array_3d[s_idx, :, :] = combined_data

            arrays_3d.append(array_3d)
    return arrays_3d


In [74]:
sample_size=60
# simulations_train = ['noise','bumps']
# simulations_test=['flat']
# val_indices = [4, 10, 11, 26, 28, 31, 33, 37] # for flat
# train_indices = [0, 1, 2, 3, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 32, 34, 35, 36, 38, 39, 40, 41] # for flat


# simulations_test=['noise']
# simulations_train = ['flat','bumps']
# val_indices = [7, 15, 17, 19, 28, 31, 32, 42, 44, 48] # for noise
# train_indices = [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 45, 46, 47] # for noise

simulations_test=['bumps']
simulations_train = ['flat','noise']
val_indices = [1, 12, 16, 18, 22, 26, 28, 37, 41] # for speedbumps
train_indices = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 17, 19, 20, 21, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 42, 43, 44] # for speedbumps

In [75]:
participants = [str(i) for i in range(1, 27)]  # Participants 101 to 123
columns_to_remove_hr = []
columns_to_remove_gsr = []
columns_to_remove_eye = ['#Frame','Time', 'Unnamed: 40','ConvergenceValid','Left_Eye_Closed','Right_Eye_Closed','LocalGazeValid','WorldGazeValid']
columns_to_remove_head = ['#Frame','Time', 'Unnamed: 14']

In [76]:
arrays_train = construct_3d_array(base_path, participants, simulations_train, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)
arrays_test = construct_3d_array(base_path, participants, simulations_test, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)

In [77]:
# Concatenate arrays along the first axis
input_train = np.concatenate(arrays_train, axis=0)
input_test = np.concatenate(arrays_test, axis=0)

# Display the shape of the final concatenated 3D array
print(f"Shape of the final concatenated 3D array: {input_train.shape}")
print(f"Shape of the final concatenated 3D array: {input_test.shape}")

Shape of the final concatenated 3D array: (83, 180, 47)
Shape of the final concatenated 3D array: (23, 180, 47)


In [78]:
def calculate_total_ssq(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    n_columns = [0, 5, 6, 7, 8, 14, 15]
    o_columns = [0, 1, 2, 3, 4, 8, 10]
    d_columns = [4, 7, 9, 10, 11, 12, 13]

    # Calculate sum for each specified set of columns
    n_val = df.iloc[:, n_columns].sum(axis=1)
    o_val = df.iloc[:, o_columns].sum(axis=1)
    d_val = df.iloc[:, d_columns].sum(axis=1)

    return n_val,o_val,d_val

In [79]:
def merge_ssq_column(conditions,participants):
  directories = []
  total_ssq_values = []
  for participant in participants:
      participant = f"{int(participant):02d}"
      for condition in conditions:
          directory = os.path.join(base_path, participant, condition)
          directories.append(directory)

  # Loop through each directory
  for directory in directories:
      # Check if the directory exists
      if not os.path.exists(directory):
          continue

      # Get all CSV files in the directory that are named 'ssq.csv'
      csv_files = [file for file in os.listdir(directory) if file == 'ssq.csv']

      # Loop through each CSV file
      for csv_file in csv_files:
          csv_path = os.path.join(directory, csv_file)
          df = pd.read_csv(csv_path)
          n_val,o_val,d_val = calculate_total_ssq(csv_path)
          total_ssq_values.append([n_val, o_val, d_val])
          #ssq_values_participant = df.iloc[:, 0:17].values.flatten()   # Assuming SSQ values are in columns 1 to 16
          #total_ssq_values.append(ssq_values_participant)
  ssq_array = np.array(total_ssq_values)
  return ssq_array

def merge_total_ssq(conditions,participants):
  directories = []
  total_ssq_values = []
  for participant in participants:
      participant = f"{int(participant):02d}"
      for condition in conditions:
          directory = os.path.join(base_path, participant, condition)
          directories.append(directory)

  # Loop through each directory
  for directory in directories:
      # Check if the directory exists
      if not os.path.exists(directory):
          continue

      # Get all CSV files in the directory that are named 'ssq.csv'
      csv_files = [file for file in os.listdir(directory) if file == 'ssq.csv']

      # Loop through each CSV file
      for csv_file in csv_files:
          csv_path = os.path.join(directory, csv_file)
          n_val,o_val,d_val = calculate_total_ssq(csv_path)
          total_ssq = (n_val+o_val+d_val) * 3.74
          df = pd.read_csv(csv_path)
          df["total-ssq"] = total_ssq
          #print("csv_path: ",csv_path,"   ",total_ssq)
          total_ssq_values.append(total_ssq)
  # Create a DataFrame from the list of total SSQ values
  df_total_ssq = pd.DataFrame(total_ssq_values, columns=["total-ssq"])
  # Convert the list of total SSQ values to a NumPy array
  total_ssq_array = np.array(total_ssq_values)
  return total_ssq_array



In [80]:
output_train=merge_ssq_column(simulations_train,participants)
output_train = np.squeeze(output_train)
output_test=merge_ssq_column(simulations_test,participants)
output_test = np.squeeze(output_test)
output_train_total_ssq=merge_total_ssq(simulations_train,participants)
output_test_total_ssq=merge_total_ssq(simulations_test,participants)
output_train_total_ssq=output_train_total_ssq.reshape(-1, 1)
output_test_total_ssq=output_test_total_ssq.reshape(-1, 1)
print(output_train.shape,output_test.shape,output_train_total_ssq.shape,output_test_total_ssq.shape)
# print(output_train)
# print(output_train_total_ssq)


(45, 3) (23, 3) (45, 1) (23, 1)


In [81]:
input_train.shape

(83, 180, 47)

In [82]:
def scale_input_data(input_train, input_test):
    # Get the shape of the input data
    num_samples_train, time_steps_train, num_features = input_train.shape
    num_samples_test, time_steps_test, _ = input_test.shape

    # Reshape the input data into 2D arrays
    flattened_train_data = input_train.reshape(-1, num_features)
    flattened_test_data = input_test.reshape(-1, num_features)

    # Initialize a MinMaxScaler object
    scaler = MinMaxScaler()

    # Fit the scaler on the training data and transform both train and test data
    scaled_train_data = scaler.fit_transform(flattened_train_data)
    scaled_test_data = scaler.transform(flattened_test_data)

    # Reshape the scaled data back to its original shape
    scaled_train_data = scaled_train_data.reshape(num_samples_train, time_steps_train, num_features)
    scaled_test_data = scaled_test_data.reshape(num_samples_test, time_steps_test, num_features)

    return scaled_train_data, scaled_test_data

def scale_target_var(target_data):
    min_val, max_val = np.min(target_data, axis=0), np.max(target_data, axis=0)
    target_data = (target_data-min_val)/(max_val-min_val)

    return target_data, min_val, max_val

In [83]:
input_train, input_test= scale_input_data(input_train[:, (60-sample_size):(180-sample_size), :], input_test[:, (60-sample_size):(180-sample_size), :])
output_train, min_val, max_val = scale_target_var(output_train)

input_val = input_train[val_indices]
input_train = input_train[train_indices]
output_val = output_train_total_ssq[val_indices]
output_train = output_train[train_indices]

print("val input :",input_val.shape)
print("train input :",input_train.shape)
print("val output :",output_val.shape)
print("train output :",output_train.shape)

val input : (9, 120, 47)
train input : (36, 120, 47)
val output : (9, 1)
train output : (36, 3)


In [84]:
input_train.shape

(36, 120, 47)

In [85]:
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense, Dropout
from keras.models import Model
import numpy as np
from keras.callbacks import EarlyStopping
import sklearn
rmse_n=[]
rmse_o=[]
rmse_d=[]
np.set_printoptions(precision=2)
for iteration in range(5):
  def get_shared_lstm(input_shape1, input_shape2):
      # Define shared LSTM model
      input_layer = Input(shape=(input_shape1, input_shape2))
      x = LSTM(64, return_sequences=False)(input_layer)
      x = Dense(256, activation='relu')(x)
      x = Dropout(0.2)(x)
      shared_model = Model(inputs=input_layer, outputs=x)
      return shared_model

  def get_output_model(shared_lstm_output, output_shape):
      # Define separate output model for each column
      output_models = []
      for i in range(output_shape[1]):
          output_model = Sequential()
          output_model.add(Dense(256, activation='relu'))
          output_model.add(Dropout(0.2))
          output_model.add(Dense(1))  # Output shape is (None, 1) for each column
          output_model_output = output_model(shared_lstm_output)
          output_models.append(output_model_output)
      return output_models

  # Assuming train_input, train_output, test_input, test_output are numpy arrays

  # Reshape train and test inputs to match LSTM input shape
  train_input_reshaped = input_train.reshape((input_train.shape[0], input_train.shape[1], input_train.shape[2]))
  test_input_reshaped = input_test.reshape((input_test.shape[0], input_test.shape[1], input_test.shape[2]))
  val_input_reshaped = input_val.reshape((input_val.shape[0], input_val.shape[1], input_val.shape[2]))

  # Get shared LSTM model
  shared_lstm = get_shared_lstm(input_train.shape[1], input_train.shape[2])

  # Create separate output models for each column
  output_models = get_output_model(shared_lstm.output, output_train.shape)

  # Create combined model
  model = Model(inputs=shared_lstm.input, outputs=output_models)

  # Compile and train the model
  model.compile(loss='mse', optimizer='adam', metrics=[['mse'] for _ in range(output_train.shape[1])])  # Using MSE as loss and metric
  early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1)
  model.fit(train_input_reshaped, output_train, epochs=20, batch_size=32,validation_data=(val_input_reshaped, output_val), callbacks=[early_stopping])


  # Predict test data
  model.fit(train_input_reshaped, [output_train[:, i] for i in range(output_train.shape[1])], epochs=1, batch_size=32)


  # Predict test data
  pred_test = np.array(model.predict(test_input_reshaped))
  print("pred_shape_before",pred_test.shape)
  pred_test = np.transpose(pred_test.squeeze(), (1, 0))
  print("pred_shape_after",pred_test.shape)

  pred_test_n = np.zeros((pred_test.shape[0], 1))
  pred_test_o = np.zeros((pred_test.shape[0], 1))
  pred_test_d = np.zeros((pred_test.shape[0], 1))
  pred_test_final = np.empty((output_test.shape[0], 0))
  for m in range(pred_test.shape[0]):
    pred_test_n[m,0] = pred_test[m,0]*(max_val[0]-min_val[0]) + min_val[0]
    pred_test_o[m,0] = pred_test[m,1]*(max_val[1]-min_val[1]) + min_val[1]
    pred_test_d[m,0] = pred_test[m,2]*(max_val[2]-min_val[2]) + min_val[2]

  pred_test_final=np.hstack((pred_test_final, pred_test_n))
  pred_test_final=np.hstack((pred_test_final, pred_test_o))
  pred_test_final=np.hstack((pred_test_final, pred_test_d))

  # Overall Test Loss
  loss_n = sklearn.metrics.mean_squared_error(pred_test_final[:,0], output_test[:,0], squared = False)
  rmse_n.append(loss_n)
  loss_o = sklearn.metrics.mean_squared_error(pred_test_final[:,1], output_test[:,1], squared = False)
  rmse_o.append(loss_o)
  loss_d= sklearn.metrics.mean_squared_error(pred_test_final[:,2], output_test[:,2], squared = False)
  rmse_d.append(loss_d)
rmse_combined = np.column_stack((rmse_n, rmse_o, rmse_d))
rmse_n_loss = sum(rmse_n) / len(rmse_n)
rmse_o_loss = sum(rmse_o) / len(rmse_o)
rmse_d_loss = sum(rmse_d) / len(rmse_d)
average_rmse = np.array([rmse_n_loss, rmse_o_loss, rmse_d_loss])

print(rmse_n_loss,rmse_o_loss,rmse_d_loss)



Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 397ms/step - loss: 0.0320 - sequential_60_mse: 0.0320 - val_loss: 5046.5454 - val_sequential_60_mse: 5046.5454
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0398 - sequential_60_mse: 0.0398 - val_loss: 5063.4526 - val_sequential_60_mse: 5063.4526
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0240 - sequential_60_mse: 0.0240 - val_loss: 5064.9287 - val_sequential_60_mse: 5064.9287
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0265 - sequential_60_mse: 0.0265 - val_loss: 5056.5425 - val_sequential_60_mse: 5056.5425
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0206 - sequential_60_mse: 0.0206 - val_loss: 5052.1680 - val_sequential_60_mse: 5052.1680
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0220 - sequential_60_mse: 0.0220 - val_loss: 5055.4683 - val_sequential_60_mse: 5055.4683
Epoch 6: early stopping
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.1015 - sequential_60_mse: 0.0248
1/1 ━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 584ms/step - loss: 0.0342 - sequential_63_mse: 0.0342 - val_loss: 5048.7227 - val_sequential_63_mse: 5048.7227
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 0.0253 - sequential_63_mse: 0.0253 - val_loss: 5069.0156 - val_sequential_63_mse: 5069.0156
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 0.0470 - sequential_63_mse: 0.0470 - val_loss: 5065.3105 - val_sequential_63_mse: 5065.3105
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - loss: 0.0498 - sequential_63_mse: 0.0498 - val_loss: 5054.2285 - val_sequential_63_mse: 5054.2285
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 0.0210 - sequential_63_mse: 0.0210 - val_loss: 5052.2700 - val_sequential_63_mse: 5052.2700
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 0.0220 - sequential_63_mse: 0.0220 - val_loss: 5057.1660 - val_sequential_63_mse: 5057.1660
Epoch 6: early stopping
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.1491 - sequential_63_mse: 0.0315
1/1 ━━

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 393ms/step - loss: 0.0493 - sequential_66_mse: 0.0493 - val_loss: 5041.8262 - val_sequential_66_mse: 5041.8262
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0547 - sequential_66_mse: 0.0547 - val_loss: 5062.6289 - val_sequential_66_mse: 5062.6289
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0228 - sequential_66_mse: 0.0228 - val_loss: 5071.1396 - val_sequential_66_mse: 5071.1396
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0476 - sequential_66_mse: 0.0476 - val_loss: 5063.9473 - val_sequential_66_mse: 5063.9473
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0313 - sequential_66_mse: 0.0313 - val_loss: 5052.6875 - val_sequential_66_mse: 5052.6875
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.0259 - sequential_66_mse: 0.0259 - val_loss: 5048.0488 - val_sequential_66_mse: 5048.0488
Epoch 6: early stopping
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 0.2102 - sequential_66_mse: 0.0355
1/1 ━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 618ms/step - loss: 0.0279 - sequential_69_mse: 0.0279 - val_loss: 5046.4624 - val_sequential_69_mse: 5046.4624
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0431 - sequential_69_mse: 0.0431 - val_loss: 5074.6299 - val_sequential_69_mse: 5074.6299
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0661 - sequential_69_mse: 0.0661 - val_loss: 5069.2793 - val_sequential_69_mse: 5069.2793
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0381 - sequential_69_mse: 0.0381 - val_loss: 5054.8374 - val_sequential_69_mse: 5054.8374
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0298 - sequential_69_mse: 0.0298 - val_loss: 5051.2710 - val_sequential_69_mse: 5051.2710
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0346 - sequential_69_mse: 0.0346 - val_loss: 5055.5103 - val_sequential_69_mse: 5055.5103
Epoch 6: early stopping
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.0614 - sequential_69_mse: 0.0264
1/1 ━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 381ms/step - loss: 0.0824 - sequential_72_mse: 0.0824 - val_loss: 5044.9873 - val_sequential_72_mse: 5044.9873
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0370 - sequential_72_mse: 0.0370 - val_loss: 5051.6729 - val_sequential_72_mse: 5051.6729
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0211 - sequential_72_mse: 0.0211 - val_loss: 5058.7676 - val_sequential_72_mse: 5058.7676
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0255 - sequential_72_mse: 0.0255 - val_loss: 5060.6772 - val_sequential_72_mse: 5060.6772
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0272 - sequential_72_mse: 0.0272 - val_loss: 5056.9126 - val_sequential_72_mse: 5056.9126
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0163 - sequential_72_mse: 0.0163 - val_loss: 5050.7520 - val_sequential_72_mse: 5050.7520
Epoch 6: early stopping
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.0834 - sequential_72_mse: 0.0241
1/1 ━━━━━━━